# VROOM testing without ORS
updated with vroom1.8.0 vroom-express 0.7, supporting Shipments

docs: https://github.com/VROOM-Project/vroom/blob/master/docs/API.md#shipments

In [32]:
import openrouteservice as ors
import requests
import folium
import json
import random
from pprint import pprint

# DATI posizione consegne
pointsA = json.load(open('./data/mezzolombardo.geojson'))['features'] #4 points
pointsB = json.load(open('./data/mezzocorona.geojson'))['features'] #6 points

vroom_url = 'http://vroom:3000/optimization/'

#centro mappa
center = [46.2147, 11.1065]
zoom = 15

# POSIZIONI PARTENZA DEI VEICOLI (Furgoni)
point_start = [46.2170, 11.1066]

# POSIZIONE ARRIVO DEI VEICOLI
point_end = [point_start[0], point_start[1]]

ccA = [p['geometry']['coordinates'] for p in pointsA]
ccB = [p['geometry']['coordinates'] for p in pointsB]

#ELENCO consegne 4 Mezzolombardo e 6 Mezzocorona TOTALE 10 con quantita'
job_locs = ccA + ccB

#colori veicoli/percorsi by vid
Vcolors = {
    '1': 'blue',
    '2': 'red',
    '3': 'green'
}

# BUILD MAP from DATA

Map = folium.Map(location=center, tiles='OpenStreetMap', zoom_start=zoom)

vehicles = []

#veicolo 1
capacity = 3
vid = 1
vehicles.append({
    'id': vid,
    'capacity': [capacity],  # Limite di capacita' del Veicolo    
    'profile': 'driving-car',
    'start': list(reversed(point_start)), # posizione di partenza veicoli
    'end': list(reversed(point_start))
})
markerText = "capacity: %s" % capacity
mark = folium.Marker(point_start, tooltip=folium.Tooltip(markerText,permanent=True), icon=folium.Icon(icon='truck', prefix='fa', color=Vcolors[str(vid)]))
mark.add_to(Map)

#veicolo 2
point_startB = [point_start[0] + 0.0015, point_start[1] - 0.0015]
capacity = 5
vid = 2
vehicles.append({
    'id': vid,
    'capacity': [capacity],  # Limite di capacita' del Veicolo    
    'profile': 'driving-car',
    'start': list(reversed(point_startB)), # posizione di partenza veicoli
    'end': list(reversed(point_startB))
})
markerText = "capacity: %s" % capacity
mark = folium.Marker(point_startB, tooltip=folium.Tooltip(markerText,permanent=True), icon=folium.Icon(icon='truck', prefix='fa', color=Vcolors[str(vid)]))
mark.add_to(Map)

jobs = []
for idx, cc in enumerate(job_locs):
    loc = list(reversed(cc))
    
    #quantita da consegnare
    delivery = random.randrange(1, 5)
    
    #tempo di della singola consegna(in secondi)
    service = 120 #random.randrange(60, 300) da 1 a 5 minuti
    
    markerText = "quantity: %s <br>time: %s min" % (delivery, round(service/60))
    mark = folium.Marker(loc, tooltip=folium.Tooltip(markerText, permanent=True), icon=folium.Icon(icon='archive', prefix='fa', color='green'))
    mark.add_to(Map)

    jobs.append({
        'id': idx,
        'location': cc,
        'delivery': [delivery], # Quantita' da consegnare
        'service': service      # tempo di consegna
    })

Map

In [33]:
request = {
    'vehicles': vehicles,
    'jobs': jobs,
    'geometry': True
}

request

{'vehicles': [{'id': 1,
   'capacity': [3],
   'profile': 'driving-car',
   'start': [11.1066, 46.217],
   'end': [11.1066, 46.217]},
  {'id': 2,
   'capacity': [5],
   'profile': 'driving-car',
   'start': [11.1051, 46.2185],
   'end': [11.1051, 46.2185]}],
 'jobs': [{'id': 0,
   'location': [11.092028617858887, 46.2173172744161],
   'delivery': [1],
   'service': 120},
  {'id': 1,
   'location': [11.08793020248413, 46.2161294935962],
   'delivery': [4],
   'service': 120},
  {'id': 2,
   'location': [11.091685295104979, 46.21057627744751],
   'delivery': [2],
   'service': 120},
  {'id': 3,
   'location': [11.096470355987549, 46.21344204466888],
   'delivery': [3],
   'service': 120},
  {'id': 4,
   'location': [11.112627983093262, 46.21758452155989],
   'delivery': [3],
   'service': 120},
  {'id': 5,
   'location': [11.118206977844238, 46.214911991592125],
   'delivery': [2],
   'service': 120},
  {'id': 6,
   'location': [11.123356819152832, 46.21419929496722],
   'delivery': [4],

In [34]:
resp = requests.post(vroom_url, json = request)

solution = resp.json()

solution['summary']
#solution

{'cost': 605,
 'unassigned': 6,
 'delivery': [8],
 'amount': [8],
 'pickup': [0],
 'service': 480,
 'duration': 605,
 'waiting_time': 0,
 'priority': 0,
 'distance': 7567,
 'computing_times': {'loading': 16, 'solving': 2, 'routing': 22}}

In [35]:
for route in solution['routes']:
    
    rtext = 'Distance: '+str(route['distance']/1000)+'km'
    rcolor = Vcolors[str(route['vehicle'])]
    print(rtext, rcolor)
        
    folium.PolyLine(
        locations = [list(reversed(coords)) for coords in ors.convert.decode_polyline(route['geometry'])['coordinates']],
        color = rcolor
    ).add_to(Map)

Map

Distance: 3.554km blue
Distance: 4.013km red


In [8]:
solution

{'code': 0,
 'summary': {'cost': 882,
  'unassigned': 5,
  'delivery': [10],
  'amount': [10],
  'pickup': [0],
  'service': 908,
  'duration': 882,
  'waiting_time': 0,
  'priority': 0,
  'distance': 10901,
  'computing_times': {'loading': 343, 'solving': 3, 'routing': 283}},
 'unassigned': [{'id': 6, 'location': [11.123356819152832, 46.21419929496722]},
  {'id': 0, 'location': [11.092028617858888, 46.2173172744161]},
  {'id': 2, 'location': [11.09168529510498, 46.21057627744751]},
  {'id': 3, 'location': [11.096470355987549, 46.21344204466888]},
  {'id': 4, 'location': [11.112627983093262, 46.21758452155989]}],
 'routes': [{'vehicle': 1,
   'cost': 404,
   'delivery': [5],
   'amount': [5],
   'pickup': [0],
   'service': 430,
   'duration': 404,
   'waiting_time': 0,
   'priority': 0,
   'distance': 4508,
   'steps': [{'type': 'start',
     'location': [11.1066, 46.217],
     'load': [5],
     'arrival': 0,
     'duration': 0,
     'distance': 0},
    {'type': 'job',
     'location'